In [70]:
# autoreload reloads modules automatically before entering the execution of code typed at the IPython prompt.
%reload_ext autoreload
%autoreload 2
%aimport feature_preprocessing_pipeline

In [92]:
from sklearn import set_config
from os.path import join, isfile
from joblib import dump, load
from feature_preprocessing_pipeline import categorize, extract_features, clean_variables, evaluate_prediction
from sklearn import tree
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [37]:
set_config(display='diagram') 

In [38]:
dataset_directory_path = 'dataset/'
model_directory_path = 'model/'

In [56]:
model_name = 'model_RMSE-678'
model_filename = f'{model_name}.joblib'
model_path = join(model_directory_path, model_filename)
model = load(model_path)

In [66]:
test_set_file = 'test_set.csv'
train_set_file = 'train_set.csv'

def get_data(file_path, target_column,dataset_directory_path='./dataset/', sample=None, callback=None):
    data = pd.read_csv(join(dataset_directory_path, file_path), index_col=0)
    if callback:
        data = callback(data)
    if sample:
        data = data.sample(n=sample)
    X = data.drop(labels=[target_column], axis=1)
    y = np.log(data[target_column])
    return X,y


## Model 

### Variables

In [57]:
rfr = model[2]
print(f"Le model {rfr} a {rfr.n_features_in_} variables")

Le model RandomForestRegressor(n_estimators=500, n_jobs=-1) a 251 variables


### Diagramme

In [58]:
model

Pipeline(steps=[('Features Extraction',
                 FunctionTransformer(func=<function extract_features at 0x7fbafb078f70>)),
                ('Columns Transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Poly features Creator',
                                                  Pipeline(steps=[('Polynomial '
                                                                   'Features',
                                                                   PolynomialFeatures(degree=3,
                                                                                      include_bias=False,
                                                                                      interaction_only=True))],
                                                           verbose=True),
                                                  <sklearn.compos...
                                                                                  unknown_value=nan))],
                                                           verbose=True),
                                                  ['transmission',
                                                   'fuel_type']),
                                                 ("'Brand' OHE",
                                                  Pipeline(steps=[('Categorizer',
                                                                   FunctionTransformer(func=<function categorize at 0x7fbb01216c10>)),
                                                                  ('OHE',
                                                                   OneHotEncoder(handle_unknown='ignore'))],
                                                           verbose=True),
                                                  ['brand'])],
                                   verbose=True)),
                ('Random Forest Regressor',
                 RandomForestRegressor(n_estimators=500, n_jobs=-1))],
         verbose=True)

## Learning Curve

![Learning curve](./Learning_curve_model_677.png)

Le premier graphique montre la courbe d'apprentissage.

Le deuxième graphique montre le temps nécessaire aux modèles pour s'entraîner avec différentes tailles de données d'entraînement. 

Le troisième graphique montre le temps nécessaire à l'entraînement des modèles pour chaque taille de données d'entraînement.

In [59]:
model[2].base_estimator_

DecisionTreeRegressor()

In [128]:
X_test,y_test = get_data(test_set_file,target_column='price',sample=25, callback=clean_variables)

In [129]:
evaluate_prediction(model, X_test, y_test)

RMSE: 6584.579507185703
prediction 	| real price
18010 		| 17998
11015 		| 18994
14400 		| 14000
17531 		| 22999
10251 		| 11249
15723 		| 22490
21404 		| 28849
17615 		| 6695
26015 		| 31999
8415 		| 10997
14577 		| 13995
9387 		| 9927
16829 		| 12794
17306 		| 36990
18630 		| 26849
12055 		| 7400
7486 		| 8298
6411 		| 6999
15279 		| 22494
8016 		| 11498
8643 		| 9689
18804 		| 28490
13008 		| 21497
8190 		| 8791
8249 		| 7998


In [130]:
tree.plot_tree(model[2].base_estimator_, 
                   #feature_names=data.feature_names,  
                   #class_names=data.target_names,
                   filled=True)

NotFittedError: This DecisionTreeRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.